In [2]:
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn import svm

from sklearn import tree

from sklearn.naive_bayes import MultinomialNB

import time

from skimage.feature import hog

# Lee los datos y guárdalos en data
data = np.load("dataset_detection/train_signs_detection.npy")
Xtrain = data[:, :-1]
ytrain = data[:, -1].reshape(-1, 1)
print(Xtrain.shape)

rn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
rn.fit(Xtrain, ytrain)

NB = MultinomialNB()
NB.fit(Xtrain, ytrain)

(701, 8100)


C:\Users\Arribas\anaconda3\envs\py37aprendizaje\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Arribas\anaconda3\envs\py37aprendizaje\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MultinomialNB()

In [3]:
import imutils
import argparse
import cv2
import time



def pyramid(image, scale=1.5, min_size=(30, 30)):
    """
    Genera imágenes sucesivamente más pequeñas con base a los parámetros proporcionados.
    :param image: Imagen base.
    :param scale: Factor de reducción de cada nivel de la pirámide.
    :param min_size: Dimensiones mínimas que debe tener un nivel de la pirámide.
    :return: Un nuevo nivel de la pirámide en cada llamada.
    """
    yield image  # El primer de la pirámide será la imagen original.

    while True:
        # Reducimos el ancho de la imagen con base a `scale`, y luego la
        # redimensionamos manteniendo la relación de aspecto.
        width = int(image.shape[1] / scale)
        image = imutils.resize(image, width=width)

        # Si el resultado es una imagen más pequeña de lo esperada, paramos.
        if image.shape[0] < min_size[1] or image.shape[1] < min_size[0]:
            break

        # Retorna el siguiente nivel de la pirámide.
        yield image


def sliding_window(image, step_size, window_size):
    """
    Genera ventanas de tamaño `window_size` hasta haber recorrido toda la imagen.
    :param image: Imagen sobre la cual generar ventanas.
    :param step_size: Número de píxeles que nos moveremos en cada dirección.
    :param window_size: Dimensiones de la ventana deslizante.
    :return: Coordenadas (x, y) de la esquina superior izquierda de la ventana, y la región como tal.
    """
    for x in range(0, image.shape[0], step_size):
        for y in range(0, image.shape[1], step_size):
            yield x, y, image[x:x + window_size[1], y:y + window_size[0]]

In [4]:
#Test de la pirámide de imágenes

# Leemos la imagen.
image = cv2.imread('Dataset_images/00001.jpg',0)

# Generamos y enumeramos cada nivel de la pirámide.
for index, level in enumerate(pyramid(image, scale=1.5)):
    # Mostramos el nivel actual.
    cv2.imshow(f'Nivel {index + 1}', level)
    print(f'Dimensiones: {level.shape}')
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Dimensiones: (800, 1360)
Dimensiones: (532, 906)
Dimensiones: (354, 604)
Dimensiones: (235, 402)
Dimensiones: (156, 268)
Dimensiones: (103, 178)
Dimensiones: (68, 118)
Dimensiones: (44, 78)


In [ ]:
def utilizarSlidingWindow(imagen,clasificador,listaPuntosRect,windowSize):
    cont = 0
    for (x,y,window) in sliding_window(imagen,int(windowSize[0]/4),int(windowSize[1]/4),windowSize)
        winW = windowSize[0]
        winH = windowSize[0]
        if window.shape[0] != winH or window.shape[1] != winW:
            continue
        window = normalizarImg(window)
        hogIm = hog(window)
        proba = svc.predict_proba(hogIm.reshape(1,-1))
        if float(proba[0,1]) >= 0.75:
            listaPuntosRect.append((x,y,x+windowSize[0],y+windowSize[0],proba[0,1]))
        return listaPuntosRect

In [47]:
#Test de la ventana deslizante

# Cargamos la imagen en memoria.
rutaImg = 'Dataset_images/00002.jpg'
image = cv2.imread(rutaImg, 0)
imagenColor = cv2.imread(rutaImg)
#Array de ventanas que han detectado una señal
detections = []
k = 1

#Iteramos cada nivel/imagen de la pirámide
for index, layer in enumerate(pyramid(image, scale=1.5)):
    print(index)
    # ... y en cada uno deslizaremos una ventana con las proporciones especificadas, moviéndonos 32 píxeles a la vez.
    for x, y, window in sliding_window(layer, step_size=32, window_size=(100, 100)):
        # Si la ventana actual no cumple con el tamaño deseado, la ignoramos.
        if window.shape!= (100,100):
            continue

        # AQUÍ ES DONDE APLICARÍAMOS EL CLASIFICADOR DE MACHINE LEARNING. SIN EMBARGO, COMO NO TENEMOS
        # NINGUNO AÚN, PINTAREMOS LA VENTANA EN LA IMAGEN.
        #acc = rn.predict_proba(hog(image[x:x+100, y:y+100]).reshape(1,-1))
        acc = NB.predict_proba(hog(window).reshape(1,-1))

        if acc[0,1] > 0.5:       
            clone = layer.copy()
            print('Coordenadas ventana recorte:\n')
            print(x,y)
            cv2.imshow(str(x)+' ', clone[x:x+100, y:y+100])

            #Guardar % comienzo x, % comienzo y, % píxeles que ocupa la ventana y acc.
            detections.append((x/layer.shape[0], y/layer.shape[1], 100/layer.shape[0], acc[0,1]))

            print('Coordenadas ventana imagen grande:\n')
            print(x,y)
            cv2.rectangle(clone, (y,x), (y + 100, x + 100), (0, 128, 0), 2)
            #cv2.rectangle(clone, (416,64), (416 + 100, 64 + 100), (0, 128, 0), 2)
            label = "{}: {:.2f}%".format('Sign', acc[0,1] * 100)

            label2 = 'Coordenadas -> x: ' + str(y) + ' y: ' + str(x)
            y2 = y - 15 if y - 15 > 15 else y + 15
            cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(clone, label2, (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            #cv2.imwrite('copiaventana' + str(k) + '.jpg', clone)
            #k = k + 1
            cv2.imshow('Ventana'+str(index), clone)   
    cv2.waitKey(0)
    cv2.destroyAllWindows()
            
detections = np.array(detections)     
clone = imagenColor.copy()

for i in range(0,detections.shape[0]):
    x = int(detections[i,0]*clone.shape[0]) #Consigo el punto donde comienza la ventana
    y = int(detections[i,1]*clone.shape[1])
    tamVentana = int(detections[i,2]*clone.shape[0])
    precision = detections[i,3]
    
    #clone = layer.copy()
    cv2.rectangle(clone, (y,x), (y + tamVentana, x + tamVentana), (0, 128, 0), 2)
    label = "{}: {:.2f}%".format('Sign', precision * 100)
    y2 = y - 20 if y - 20 > 15 else y + 15
    cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,128,0), 2)
    
#cv2.imwrite('copiaresultadofinal.jpg', image)

cv2.imshow('Ventana', clone) 
cv2.waitKey(0)
cv2.destroyAllWindows()
for i in detections:
    print(i)

0
Coordenadas ventana recorte:

448 960
Coordenadas ventana imagen grande:

448 960
Coordenadas ventana recorte:

448 992
Coordenadas ventana imagen grande:

448 992
Coordenadas ventana recorte:

448 1024
Coordenadas ventana imagen grande:

448 1024
Coordenadas ventana recorte:

480 992
Coordenadas ventana imagen grande:

480 992
Coordenadas ventana recorte:

480 1024
Coordenadas ventana imagen grande:

480 1024
1
Coordenadas ventana recorte:

288 640
Coordenadas ventana imagen grande:

288 640
Coordenadas ventana recorte:

320 640
Coordenadas ventana imagen grande:

320 640
2
3
4
5
6
7
[0.56       0.70588235 0.125      0.98236948]
[0.56       0.72941176 0.125      0.9999999 ]
[0.56       0.75294118 0.125      1.        ]
[0.6        0.72941176 0.125      0.99999972]
[0.6        0.75294118 0.125      0.99999997]
[0.54135338 0.70640177 0.18796992 0.99999554]
[0.60150376 0.70640177 0.18796992 0.98830201]
